### Simple motion play of `Common Rig` 

In [1]:
import pickle, os
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r, quat2r
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig.xml`

In [2]:
xml_path = '../asset/smpl_rig/scene_smpl_rig.xml'
env = MuJoCoParserClass(name='Common Rig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0083] HZ:[120]
n_dof (=nv):[43]
n_geom:[67]
geom_names:['floor', 'base', 'base2lpelvis', 'base2rpelvis', 'base2spine1', 'spine2spine', 'spine2spine2', 'spine2spine3', 'spine2lcollar', 'spine2rcollar', 'neck2head', 'head', 'nose', 'lcollar2lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2lindex1', 'lwrist2lmiddle1', 'lwrist2lpinky1', 'lwrist2lring1', 'lwrist2lthumb1', 'lindex1-lindex2', 'lindex2-lindex3', 'lindex3-lindextip', 'lmiddle1-lmiddle2', 'lmiddle2-lmiddle3', 'lmiddl3-lmiddletip', 'lring1-lring2', 'lring2-lring3', 'lring3-lringtip', 'lpinky1-lpinky2', 'lpinky2-lpinky3', 'lpinky3-lpinkytip', 'lthumb1-lthumb2', 'lthumb2-lthumb3', 'lthumb3-lthumbtip', 'rcollar2rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rindex1', 'rwrist2rmiddle1', 'rwrist2rpinky1', 'rwrist2rring1', 'rwrist2rthumb1', 'rindex1-rindex2', 'rindex2-rindex3', 'rindex3-rindextip', 'rmiddle1-rmiddle2', 'rmiddle2-rmiddle3', 'rmiddle3-rmiddletip', 'rring1-rring2', 'rring2-rring3', 'rring3-rringtip

In [3]:
env.body_names

['world',
 'base',
 'root',
 'spine1',
 'spine2',
 'spine3',
 'neck',
 'head',
 'left_collar',
 'left_shoulder',
 'left_elbow',
 'left_wrist',
 'left_index1',
 'left_index2',
 'left_index3',
 'left_index4',
 'left_middle1',
 'left_middle2',
 'left_middle3',
 'left_middle4',
 'left_ring1',
 'left_ring2',
 'left_ring3',
 'left_ring4',
 'left_pinky1',
 'left_pinky2',
 'left_pinky3',
 'left_pinky4',
 'left_thumb1',
 'left_thumb2',
 'left_thumb3',
 'left_thumb4',
 'right_collar',
 'right_shoulder',
 'right_elbow',
 'right_wrist',
 'right_index1',
 'right_index2',
 'right_index3',
 'right_index4',
 'right_middle1',
 'right_middle2',
 'right_middle3',
 'right_middle4',
 'right_ring1',
 'right_ring2',
 'right_ring3',
 'right_ring4',
 'right_pinky1',
 'right_pinky2',
 'right_pinky3',
 'right_pinky4',
 'right_thumb1',
 'right_thumb2',
 'right_thumb3',
 'right_thumb4',
 'left_pelvis',
 'left_knee',
 'left_ankle',
 'right_pelvis',
 'right_knee',
 'right_ankle']

In [4]:
# Set which joints to control
rev_joint_names = env.rev_joint_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=rev_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=rev_joint_names)
q_rev_init = env.get_qpos_joints(rev_joint_names)
n_rev_joint = len(rev_joint_names)

### Load motion and playback

In [5]:
motion_file_path = os.path.join("../asset/smpl_rig/motion/recon_smpl_walk.npy")
device='cpu'

In [6]:
# from amp.utils.motion_lib import MotionLib
from amp.tasks.common_rig_amp import MotionLib
KEY_BODY_NAMES = ["right_ankle", "left_ankle", "right_wrist", "left_wrist"]
key_body_ids = np.array(env.get_body_ids(KEY_BODY_NAMES))
motion_lib = MotionLib(motion_file=motion_file_path,
                       num_dofs=env.n_rev_joint,
                       key_body_ids=key_body_ids,
                       device=device)
motion_lib.get_motion(0)
qs = motion_lib.get_motion(0).qpos
p_roots = motion_lib.get_motion(0).root_translation
# p_roots += 0.05
quat_roots = motion_lib.get_motion(0).global_root_rotation
L = qs.shape[0]

/Users/yoonbyung/opt/anaconda3/envs/snapbot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-25 18:18:10,196	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-25 18:18:10,323 - INFO - logger - logger initialized
<unknown>:3: DeprecationWarning: invalid escape sequence \*


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html
Loading 1/1 motion files: ../asset/smpl_rig/motion/recon_smpl_walk.npy
Loaded 1 motions with a total length of 3.908s.


In [8]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='SMPL Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0

while env.is_viewer_alive():
    root_pos, root_rot, dof_pos, root_vel, root_ang_vel, dof_vel, key_pos = motion_lib.get_motion_state([0], env.tick * env.dt)
    # Update 
    quat_root = quat_roots[tick,:] # [4] quaternion
    if tick==(L-1): tick = 0
    else: tick = tick + 1
    env.set_p_root(root_name='base',p=root_pos)
    env.set_quat_root(root_name='base',quat=root_rot[:,[3,0,1,2]])
    env.forward(q=dof_pos,joint_idxs=joint_idxs_fwd)
    
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='foot1_right',axis_len=0.3)
        env.plot_geom_T(geom_name='foot1_left',axis_len=0.3)
        # Plot revolute joints with arrow
        # env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        # Plot body
        
        for pos in key_pos[0]:
            env.plot_T(p=pos.numpy(), R=np.eye(3,3),
                    PLOT_AXIS=True,axis_len=0.1, axis_width=0.005)
        # r_ = motion_lib.get_motion(0).global_rotation
        # t_ = motion_lib.get_motion(0).xpos
        # for i in range(env.n_body-1):
        #     env.plot_T(p=t_[tick, i], R=quat2r(r_[tick, i]),
        #             PLOT_AXIS=True,axis_len=0.1, axis_width=0.005)
        env.render()
        
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

Done.


In [8]:
pos.numpy()

array([3.82, 0.17, 0.86], dtype=float32)

### First, get the sequence of two feet pose

In [9]:
p_rfoot_traj = np.zeros((L,3))
p_lfoot_traj = np.zeros((L,3))
for tick in range(L):
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    env.set_p_root(root_name='base',p=p_root)
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    # Append
    p_rfoot_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
    p_lfoot_traj[tick,:] = env.get_p_geom(geom_name='lfoot')

NameError: name 'sample_qs' is not defined

: 

In [ ]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0
p_rfoot_centered_traj = np.zeros((L,3))
p_lfoot_centered_traj = np.zeros((L,3))
while env.is_viewer_alive():
    
    # Update
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
    env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    # Append translated foot traj
    p_rfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
    p_lfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='lfoot')
    
    # Render
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
        
    # Increase tick
    if tick==(L-1): tick = 0
    else: tick = tick + 1
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

### Solve IK for two feet

In [ ]:
p_trgt_rfoot = np.average(p_rfoot_centered_traj,axis=0)
p_trgt_lfoot = np.average(p_lfoot_centered_traj,axis=0)
R_trgt_rfoot = rpy2r(np.radians([0,0,0]))
R_trgt_lfoot = rpy2r(np.radians([0,0,0]))
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
while env.is_viewer_alive():
    # Update
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
    env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    # Solve IK
    ik_geom_names = ['rfoot','lfoot']
    ik_p_trgts = [p_trgt_rfoot,p_trgt_lfoot]
    ik_R_trgts = [R_trgt_rfoot,R_trgt_lfoot]
    err_traj = np.zeros(30)
    for ik_tick in range(30):
        J_list,ik_err_list = [],[]
        for ik_idx,ik_geom_name in enumerate(ik_geom_names):
            ik_p_trgt = ik_p_trgts[ik_idx]
            ik_R_trgt = ik_R_trgts[ik_idx]
            J,ik_err = env.get_ik_ingredients_geom(
                geom_name=ik_geom_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                IK_P=True,IK_R=True)
            J_list.append(J)
            ik_err_list.append(ik_err)
        J_stack      = np.vstack(J_list)
        ik_err_stack = np.hstack(ik_err_list)
        err_traj[ik_tick] = np.max(np.abs(ik_err_stack))
        dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
        q = q + dq[joint_idxs_jac]
        env.forward(q=q,joint_idxs=joint_idxs_fwd)
        
    # Render
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
        
    # Increase tick
    if tick==(L-1): tick = 0
    else: tick = tick + 1
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

Done.
